# Agno Agent Tutorial for Engineers
## Advanced Guide to Building AI Agents

This tutorial covers building sophisticated AI agents using Agno, including:
- Agent architecture and configuration
- Memory and session management
- Tool integration and custom toolkits
- Workflows and multi-agent systems
- Advanced patterns and best practices

## Setup and Installation

In [ ]:
# Install Agno
!pip install agno openai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set your API keys 
os.environ["OPENAI_API_KEY"] = "your-api-key-here"

## Part 1: Basic Agent Creation

Let's start with a simple agent and progressively add complexity.

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

# Create a basic agent
basic_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="BasicAssistant",
    description="A helpful AI assistant",
    instructions=[
        "You are a helpful assistant.",
        "Provide clear and concise answers.",
        "Ask for clarification when needed."
    ],
    markdown=True,
    debug_mode=True
)

# Test the agent
basic_agent.print_response("What is machine learning?")

## Part 2: Adding Tools and Toolkits

Tools allow agents to perform actions and retrieve information.

In [ ]:
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools

# Create an agent with multiple toolkits
research_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="ResearchAgent",
    description="An agent that can search the web and analyze financial data",
    instructions=[
        "Use web search to find current information.",
        "Use financial tools to get stock data.",
        "Always cite your sources.",
        "Provide data-driven insights."
    ],
    tools=[
        DuckDuckGoTools(
            enable_search=True,
            enable_news=True,
            fixed_max_results=5
        ),
        YFinanceTools()
    ],
    markdown=True,
    show_tool_calls=True
)

# Test with a complex query
research_agent.print_response(
    "What's the latest news about Tesla and what's their current stock price?"
)

## Part 3: Custom Tools

Create your own tools for specific functionality.

In [ ]:
from typing import Optional
import requests

# Define custom tools as functions
def get_weather(city: str, units: str = "metric") -> str:
    """Get current weather for a city.
    
    Args:
        city: The city name
        units: Temperature units (metric or imperial)
    
    Returns:
        Weather information as a string
    """
    # This is a placeholder - replace with actual API call
    return f"Weather in {city}: 22°C, Partly cloudy"

def calculate_roi(initial_investment: float, final_value: float) -> dict:
    """Calculate return on investment.
    
    Args:
        initial_investment: Initial investment amount
        final_value: Final value of investment
    
    Returns:
        Dictionary with ROI percentage and profit
    """
    profit = final_value - initial_investment
    roi_percentage = (profit / initial_investment) * 100
    
    return {
        "profit": round(profit, 2),
        "roi_percentage": round(roi_percentage, 2),
        "initial_investment": initial_investment,
        "final_value": final_value
    }

# Create agent with custom tools
custom_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="CustomToolAgent",
    tools=[get_weather, calculate_roi],
    instructions=["Use the available tools to answer user questions."],
    show_tool_calls=True,
    markdown=True
)

custom_agent.print_response(
    "What's the weather in London? Also, if I invested $10,000 and now have $15,000, what's my ROI?"
)

## Part 4: Memory and Session Management

Enable agents to remember context across conversations.

In [ ]:
from agno.storage.sqlite import SqliteStorage

# Create agent with memory
memory_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="MemoryAgent",
    storage=SqliteStorage(table_name="agent_sessions", db_file="agent_memory.db"),
    
    # Session configuration
    user_id="engineer_001",
    session_id="tutorial_session",
    
    # Memory settings
    add_history_to_context=True,
    num_history_messages=10,
    
    # Enable user memories
    enable_user_memories=True,
    add_memories_to_context=True,
    
    # Session state
    session_state={"conversation_count": 0, "topics": []},
    add_session_state_to_context=True,
    
    markdown=True
)

# Have a multi-turn conversation
print("=== Turn 1 ===")
memory_agent.print_response("My name is Alex and I'm interested in AI agents.")

print("\n=== Turn 2 ===")
memory_agent.print_response("What did I just tell you about myself?")

print("\n=== Turn 3 ===")
memory_agent.print_response("Can you remind me what we've discussed?")

## Part 5: Knowledge Base Integration (RAG)

Add a knowledge base for retrieval-augmented generation.

In [ ]:
from agno.knowledge.pdf import PDFKnowledgeBase, PDFReader
from agno.vectordb.lancedb import LanceDb
from agno.embedder.openai import OpenAIEmbedder

# Create a knowledge base from PDFs
knowledge_base = PDFKnowledgeBase(
    path="docs/",  # Path to your PDF files
    vector_db=LanceDb(
        table_name="pdf_documents",
        uri="./lancedb",
        embedder=OpenAIEmbedder(model="text-embedding-3-small")
    ),
    reader=PDFReader(chunk=True)
)

# Load the knowledge base (only need to do this once)
# knowledge_base.load(recreate=False)

# Create agent with knowledge
rag_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="RAGAgent",
    knowledge=knowledge_base,
    
    # RAG settings
    search_knowledge=True,
    add_knowledge_to_context=True,
    references_format="yaml",
    
    instructions=[
        "Use the knowledge base to answer questions.",
        "Always cite specific documents when referencing information.",
        "If information isn't in the knowledge base, say so."
    ],
    markdown=True,
    show_tool_calls=True
)

rag_agent.print_response("What information do we have about AI safety?")

## Part 6: Structured Outputs with Pydantic

Get structured, validated responses from agents.

In [ ]:
from pydantic import BaseModel, Field
from typing import List

# Define output schema
class CompanyAnalysis(BaseModel):
    company_name: str = Field(..., description="Name of the company")
    ticker: str = Field(..., description="Stock ticker symbol")
    current_price: float = Field(..., description="Current stock price")
    market_cap: str = Field(..., description="Market capitalization")
    key_strengths: List[str] = Field(..., description="Top 3-5 strengths")
    key_risks: List[str] = Field(..., description="Top 3-5 risks")
    recommendation: str = Field(..., description="Buy/Hold/Sell recommendation")
    reasoning: str = Field(..., description="Reasoning for the recommendation")

# Create agent with structured output
structured_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="StructuredAnalyst",
    tools=[YFinanceTools()],
    output_model=CompanyAnalysis,
    structured_outputs=True,
    instructions=[
        "Analyze the requested company thoroughly.",
        "Use financial tools to get accurate data.",
        "Provide balanced analysis with both strengths and risks."
    ]
)

# Get structured response
response = structured_agent.run("Analyze Apple Inc. (AAPL)")
analysis: CompanyAnalysis = response.content

print(f"Company: {analysis.company_name}")
print(f"Ticker: {analysis.ticker}")
print(f"Price: ${analysis.current_price}")
print(f"\nStrengths:\n" + "\n".join(f"- {s}" for s in analysis.key_strengths))
print(f"\nRisks:\n" + "\n".join(f"- {r}" for r in analysis.key_risks))
print(f"\nRecommendation: {analysis.recommendation}")
print(f"\nReasoning: {analysis.reasoning}")

## Part 7: Reasoning Agents

Enable step-by-step reasoning for complex problems.

In [ ]:
from agno.tools.reasoning import ReasoningTools

# Create reasoning agent
reasoning_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="ReasoningAgent",
    
    # Enable reasoning
    reasoning=True,
    reasoning_min_steps=3,
    reasoning_max_steps=10,
    
    # Add reasoning tools
    tools=[ReasoningTools(think=True, analyze=True)],
    
    instructions=[
        "Break down complex problems into steps.",
        "Think through each step carefully.",
        "Show your reasoning process."
    ],
    markdown=True,
    show_tool_calls=True
)

reasoning_agent.print_response(
    """A farmer has 17 sheep. All but 9 die. How many sheep are left?
    Please think through this step by step."""
)

## Part 8: Workflows - Multi-Step Agent Orchestration

Create complex workflows with multiple agents working together.

In [ ]:
from agno.workflow import Workflow, RunResponse
from agno.tools.tavily import TavilyTools

# Define specialized agents
researcher = Agent(
    name="Researcher",
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools()],
    instructions=["Research the topic thoroughly.", "Gather factual information."]
)

analyst = Agent(
    name="Analyst",
    model=OpenAIChat(id="gpt-4o"),
    instructions=[
        "Analyze the research data.",
        "Identify trends and patterns.",
        "Provide insights and recommendations."
    ]
)

writer = Agent(
    name="Writer",
    model=OpenAIChat(id="gpt-4o"),
    instructions=[
        "Write a comprehensive report.",
        "Use clear and professional language.",
        "Include executive summary and key findings."
    ],
    markdown=True
)

# Define workflow steps
def research_workflow(input_data: str) -> RunResponse:
    """Multi-step research workflow."""
    
    # Step 1: Research
    research_result = researcher.run(f"Research: {input_data}")
    
    # Step 2: Analyze
    analysis_result = analyst.run(
        f"Analyze this research:\n{research_result.content}"
    )
    
    # Step 3: Write report
    report = writer.run(
        f"""Write a comprehensive report based on:
        
        Research:
        {research_result.content}
        
        Analysis:
        {analysis_result.content}
        """
    )
    
    return report

# Create and run workflow
workflow = Workflow(
    name="ResearchWorkflow",
    steps=research_workflow,
    description="Research, analyze, and report on any topic"
)

workflow.print_response("Latest developments in quantum computing")

## Part 9: Advanced Features - Hooks and Guardrails

Add pre/post processing and guardrails to your agents.

In [ ]:
from agno.run import RunResponse
import re

# Define pre-hook (runs before agent processes input)
def input_validator(agent: Agent, input_data: str) -> str:
    """Validate and sanitize input."""
    print(f"[PRE-HOOK] Validating input: {input_data[:50]}...")
    
    # Example: Check for sensitive patterns
    if re.search(r'\b\d{3}-\d{2}-\d{4}\b', input_data):
        print("[WARNING] Potential SSN detected, removing...")
        input_data = re.sub(r'\b\d{3}-\d{2}-\d{4}\b', '[REDACTED]', input_data)
    
    return input_data

# Define post-hook (runs after agent generates output)
def output_filter(agent: Agent, response: RunResponse) -> RunResponse:
    """Filter and enhance output."""
    print(f"[POST-HOOK] Processing output...")
    
    # Add disclaimer
    response.content += "\n\n---\n*This response was generated by AI. Please verify important information.*"
    
    return response

# Define tool hook (runs between tool calls)
def tool_monitor(agent: Agent, tool_name: str, tool_result: any) -> any:
    """Monitor tool usage."""
    print(f"[TOOL-HOOK] Tool '{tool_name}' executed successfully")
    return tool_result

# Create agent with hooks
guarded_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="GuardedAgent",
    tools=[DuckDuckGoTools()],
    
    # Add hooks
    pre_hooks=[input_validator],
    post_hooks=[output_filter],
    tool_hooks=[tool_monitor],
    
    markdown=True
)

guarded_agent.print_response("Search for information about AI safety")

## Part 10: Streaming Responses

Stream agent responses in real-time.

In [ ]:
# Create streaming agent
streaming_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    name="StreamingAgent",
    stream=True,
    stream_events=True,
    markdown=True
)

# Stream with print_response
print("=== Streaming with print_response ===")
streaming_agent.print_response(
    "Write a short story about a robot learning to code",
    stream=True
)

# Manual streaming for more control
print("\n\n=== Manual streaming ===")
for chunk in streaming_agent.run("Explain machine learning in simple terms", stream=True):
    if chunk.content:
        print(chunk.content, end="", flush=True)

## Part 11: Production-Ready Agent Template

A comprehensive example with all production features.

In [ ]:
from datetime import datetime

# Production agent with all features
production_agent = Agent(
    # Core configuration
    model=OpenAIChat(id="gpt-4o"),
    name="ProductionAgent",
    description="A production-ready AI agent with full feature set",
    
    # Instructions
    instructions=[
        "You are a professional AI assistant.",
        "Always prioritize accuracy over speed.",
        "Ask clarifying questions when needed.",
        "Cite sources for factual claims.",
        "Admit when you don't know something."
    ],
    
    # Tools
    tools=[
        DuckDuckGoTools(enable_search=True, enable_news=True),
        YFinanceTools(),
    ],
    tool_call_limit=10,
    
    # Memory and storage
    storage=SqliteStorage(table_name="prod_sessions", db_file="production.db"),
    add_history_to_context=True,
    num_history_messages=20,
    enable_user_memories=True,
    add_memories_to_context=True,
    
    # Session management
    user_id="production_user",
    session_id=f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    add_session_state_to_context=True,
    
    # Context
    add_datetime_to_context=True,
    add_name_to_context=True,
    markdown=True,
    
    # Error handling
    retries=3,
    delay_between_retries=2,
    exponential_backoff=True,
    
    # Debugging and monitoring
    debug_mode=False,
    telemetry=True,
    show_tool_calls=True,
    
    # Performance
    stream=False,
    cache_session=True,
)

# Use the production agent
production_agent.print_response(
    "Analyze the current AI market trends and provide investment recommendations"
)

## Part 12: Best Practices and Tips

### 1. **Model Selection**
- Use `gpt-4o` for complex reasoning and tool use
- Use `gpt-4o-mini` for faster, cheaper responses
- Consider Claude for long context tasks

### 2. **Memory Management**

# Good: Limit history for cost control
num_history_messages=10

# Good: Use session state for persistent data
session_state={"user_preferences": {}, "context": {}}


### 3. **Tool Design**

# Good: Clear docstrings and type hints
def my_tool(param: str, optional: int = 10) -> dict:
    """Clear description of what the tool does.
    
    Args:
        param: Description of param
        optional: Description with default
    
    Returns:
        Description of return value
    """
    pass


### 4. **Error Handling**

# Always use retries in production
retries=3
exponential_backoff=True

# Wrap agent calls in try-except
try:
    response = agent.run(query)
except Exception as e:
    logger.error(f"Agent error: {e}")
    # Handle gracefully


### 5. **Cost Optimization**
- Use `num_history_messages` to limit context
- Set `tool_call_limit` to prevent runaway executions
- Cache results when appropriate
- Use cheaper models for simple tasks

### 6. **Testing**

# Test with debug mode
agent.debug_mode = True

# Test edge cases
test_cases = [
    "empty input: ",
    "very long input: " + "x" * 10000,
    "special characters: !@#$%^&*()",
]


## Exercises

### Exercise 1: Build a Code Review Agent
Create an agent that reviews Python code and provides suggestions.

### Exercise 2: Multi-Agent Research System
Build a workflow with:
- Research agent (gathers information)
- Fact-checker agent (verifies claims)
- Writer agent (creates report)

### Exercise 3: Customer Support Agent
Create an agent with:
- Knowledge base of FAQs
- Memory of customer interactions
- Ability to escalate to human

### Exercise 4: Data Analysis Agent
Build an agent that:
- Accepts CSV files
- Analyzes data
- Returns structured insights (Pydantic model)
- Generates visualizations

## Additional Resources

- [Agno Documentation](https://docs.agno.com)
- [GitHub Repository](https://github.com/agno-agi/agno)
- [Community Discord](https://discord.gg/agno)
- [Example Projects](https://github.com/agno-agi/examples)

## Next Steps

1. Experiment with different model providers
2. Build custom toolkits for your domain
3. Explore advanced workflows and teams
4. Implement production monitoring and logging
5. Contribute to the Agno community!